In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-sep-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-sep-2022/train.csv
/kaggle/input/tabular-playground-series-sep-2022/test.csv


In [2]:
import math

# Set up train and validate dfs

In [3]:
df_raw = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2022/train.csv')

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70128 entries, 0 to 70127
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   row_id    70128 non-null  int64 
 1   date      70128 non-null  object
 2   country   70128 non-null  object
 3   store     70128 non-null  object
 4   product   70128 non-null  object
 5   num_sold  70128 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 3.2+ MB


In [5]:
train_size = math.ceil(df_raw.shape[0] * .80)

In [6]:
train_size

56103

In [7]:
df_train = df_raw.head(train_size)

In [8]:
df_validate = df_raw.tail(len(df_raw) - train_size)

In [9]:
assert len(df_train) + len(df_validate) == len(df_raw)

# EDA

In [10]:
df_train.head()

,row_id,date,country,store,product,num_sold
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240


In [11]:
df_train.describe()

,row_id,num_sold
count,56103.000000,56103.000000
mean,28051.000000,189.180525
std,16195.685413,126.980382
min,0.000000,19.000000
25%,14025.500000,91.000000
50%,28051.000000,145.000000
75%,42076.500000,273.000000
max,56102.000000,986.000000


In [12]:
df_grouped = df_train.groupby(['country', 'store', 'product'])

In [13]:
df_grouped.describe().drop(columns = ['row_id'])

num_sold              \
                                                       count        mean   
country store      product                                                 
Belgium KaggleMart Kaggle Advanced Techniques         1169.0  419.727117   
                   Kaggle Getting Started             1169.0  331.745937   
                   Kaggle Recipe Book                 1169.0  250.982891   
                   Kaggle for Kids: One Smart Goose   1169.0  444.709153   
        KaggleRama Kaggle Advanced Techniques         1169.0  145.734816   
                   Kaggle Getting Started             1169.0  115.124893   
                   Kaggle Recipe Book                 1169.0   87.209581   
                   Kaggle for Kids: One Smart Goose   1169.0  154.707442   
France  KaggleMart Kaggle Advanced Techniques         1169.0  370.952951   
                   Kaggle Getting Started             1169.0  292.045338   
                   Kaggle Recipe Book                 1169.0  221.171086   
                   Kaggle for Kids: One Smart Goose   1169.0  393.396065   
        KaggleRama Kaggle Advanced Techniques         1169.0  128.480753   
                   Kaggle Getting Started             1169.0  101.167665   
                   Kaggle Recipe Book                 1169.0   76.563730   
                   Kaggle for Kids: One Smart Goose   1169.0  136.189906   
Germany KaggleMart Kaggle Advanced Techniques         1169.0  420.204448   
                   Kaggle Getting Started             1169.0  332.138580   
                   Kaggle Recipe Book                 1169.0  252.001711   
                   Kaggle for Kids: One Smart Goose   1169.0  447.198460   
        KaggleRama Kaggle Advanced Techniques         1169.0  146.589393   
                   Kaggle Getting Started             1169.0  115.630453   
                   Kaggle Recipe Book                 1169.0   87.292558   
                   Kaggle for Kids: One Smart Goose   1169.0  155.078700   
Italy   KaggleMart Kaggle Advanced Techniques         1169.0  313.872541   
                   Kaggle Getting Started             1169.0  246.514115   
                   Kaggle Recipe Book                 1169.0  187.089820   
                   Kaggle for Kids: One Smart Goose   1169.0  332.126604   
        KaggleRama Kaggle Advanced Techniques         1169.0  108.680924   
                   Kaggle Getting Started             1169.0   85.557742   
                   Kaggle Recipe Book                 1169.0   64.837468   
                   Kaggle for Kids: One Smart Goose   1169.0  114.765612   
Poland  KaggleMart Kaggle Advanced Techniques         1169.0  155.667237   
                   Kaggle Getting Started             1169.0  120.490163   
                   Kaggle Recipe Book                 1169.0   92.692044   
                   Kaggle for Kids: One Smart Goose   1169.0  163.384089   
        KaggleRama Kaggle Advanced Techniques         1169.0   53.823781   
                   Kaggle Getting Started             1169.0   41.578272   
                   Kaggle Recipe Book                 1169.0   31.911035   
                   Kaggle for Kids: One Smart Goose   1168.0   56.599315   
Spain   KaggleMart Kaggle Advanced Techniques         1168.0  277.574486   
                   Kaggle Getting Started             1168.0  218.343322   
                   Kaggle Recipe Book                 1168.0  165.882705   
                   Kaggle for Kids: One Smart Goose   1168.0  292.547089   
        KaggleRama Kaggle Advanced Techniques         1168.0   96.103596   
                   Kaggle Getting Started             1168.0   75.303938   
                   Kaggle Recipe Book                 1168.0   57.221747   
                   Kaggle for Kids: One Smart Goose   1168.0  101.744863   

                                                                              \
                                                           std    min    25%   
country store  

In [14]:
df_grouped.size().to_frame().rename(columns = {0: 'num_rows'}).reset_index()

,country,store,product,num_rows
0,Belgium,KaggleMart,Kaggle Advanced Techniques,1169
1,Belgium,KaggleMart,Kaggle Getting Started,1169
2,Belgium,KaggleMart,Kaggle Recipe Book,1169
3,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,1169
4,Belgium,KaggleRama,Kaggle Advanced Techniques,1169
5,Belgium,KaggleRama,Kaggle Getting Started,1169
6,Belgium,KaggleRama,Kaggle Recipe Book,1169
7,Belgium,KaggleRama,Kaggle for Kids: One Smart Goose,1169
8,France,KaggleMart,Kaggle Advanced Techniques,1169
9,France,KaggleMart,Kaggle Getting Started,1169


In [15]:
df_grouped.apply(lambda row: (min(row.date), max(row.date))).to_frame().rename(columns = {0: 'min_max_dates'}).reset_index()

,country,store,product,min_max_dates
0,Belgium,KaggleMart,Kaggle Advanced Techniques,"(2017-01-01, 2020-03-14)"
1,Belgium,KaggleMart,Kaggle Getting Started,"(2017-01-01, 2020-03-14)"
2,Belgium,KaggleMart,Kaggle Recipe Book,"(2017-01-01, 2020-03-14)"
3,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,"(2017-01-01, 2020-03-14)"
4,Belgium,KaggleRama,Kaggle Advanced Techniques,"(2017-01-01, 2020-03-14)"
5,Belgium,KaggleRama,Kaggle Getting Started,"(2017-01-01, 2020-03-14)"
6,Belgium,KaggleRama,Kaggle Recipe Book,"(2017-01-01, 2020-03-14)"
7,Belgium,KaggleRama,Kaggle for Kids: One Smart Goose,"(2017-01-01, 2020-03-14)"
8,France,KaggleMart,Kaggle Advanced Techniques,"(2017-01-01, 2020-03-14)"
9,France,KaggleMart,Kaggle Getting Started,"(2017-01-01, 2020-03-14)"


# Feature engineering

In [16]:
def feature_step_01(x: pd.Series) -> pd.Series: 
    pass 

def feature_step_02(x: pd.Series) -> pd.Series: 
    pass

# Training config

In [17]:
class model_01_average_last_x_months: 
    def __init__(self, last_x_months): 
        self.last_x_months = last_x_months
        
    def fit(target: np.array): 
        return np.mean(target)
    
    def predict(): 
        pass 
        

models = {
    1: ('average of last x months', 6),
    2: '<model_02>',
}

# Training

In [18]:
def train_model(model_num): 
    # read in data 
    
    # feature engineering 
    
    # initialize model:
    m = models.get(model_num)
    
    # fit model: 
    
    
    
    pass